## CBC - Instâncias A (números inteiros)

In [1]:
using JuMP
using DelimitedFiles
using Cbc
using MathOptInterface
using CSV
using DataFrames

const MOI = MathOptInterface

# Criando os arrays que irão armazenar os dados de resolução de cada instância
fileName = []
agentsNumber = []
solveTime = []
isOptimalSolution = []
solution = []

# Obtendo o nome de todas as instâncias
files = readdir("instancias/insta"; join=true)

105-element Vector{String}:
 "instancias/insta/inst_1_100.txt"
 "instancias/insta/inst_1_1000.txt"
 "instancias/insta/inst_1_200.txt"
 "instancias/insta/inst_1_50.txt"
 "instancias/insta/inst_1_500.txt"
 "instancias/insta/insta_2_10_100.txt"
 "instancias/insta/insta_2_10_1000.txt"
 "instancias/insta/insta_2_10_200.txt"
 "instancias/insta/insta_2_10_50.txt"
 "instancias/insta/insta_2_10_500.txt"
 "instancias/insta/insta_2_1_100.txt"
 "instancias/insta/insta_2_1_1000.txt"
 "instancias/insta/insta_2_1_200.txt"
 ⋮
 "instancias/insta/insta_3_7_50.txt"
 "instancias/insta/insta_3_7_500.txt"
 "instancias/insta/insta_3_8_100.txt"
 "instancias/insta/insta_3_8_1000.txt"
 "instancias/insta/insta_3_8_200.txt"
 "instancias/insta/insta_3_8_50.txt"
 "instancias/insta/insta_3_8_500.txt"
 "instancias/insta/insta_3_9_100.txt"
 "instancias/insta/insta_3_9_1000.txt"
 "instancias/insta/insta_3_9_200.txt"
 "instancias/insta/insta_3_9_50.txt"
 "instancias/insta/insta_3_9_500.txt"

In [2]:
# Iterando sob cada instância
for f in files

    # Criando o modelo que usa o CBC solver
    model = Model(Cbc.Optimizer)
    set_optimizer_attribute(model, "seconds", 60 * 30)
    set_optimizer_attribute(model, "Heuristics", "off")
    set_optimizer_attribute(model, "logLevel", 0)

    # Lendo a matriz de custo
    c = readdlm(f, Int)

    # Checando o tamanho das linhas da matriz
    n = size(c)[1]

    # Definindo variável de decisão -> Xij = {0,1}
    @variable(model,x[i=1:n,j=1:n], Bin)

    # Função objetivo
    @objective(model, Max, sum(c[i, j] * x[i, j] for i in 1:n, j in 1:n))

    # Restrições para garantir a saída de todo vértice
    for i in 1:n
        @constraint(model,sum(x[i,j] for j in 1 : n) == 1)
    end

    # Restrições para garantir a chegada em cada vértice
    for j in 1:n
        @constraint(model,sum(x[i,j] for i in 1:n) == 1)
    end

    # Trazendo uma solução otima
    JuMP.optimize!(model)

    # Adicionando nos arrays os valores referentes a instância em questão e sua solução.
    push!(fileName, f)
    push!(agentsNumber, n)
    push!(solveTime, solve_time(model))
    push!(isOptimalSolution, termination_status(model) == MOI.OPTIMAL)
    push!(solution, objective_value(model))
end

row inf 1.46549e-14
column inf 2.53531e-08
1 slacks added
1 slacks added
row inf 2.05391e-14
column inf 3.11478e-08
1 slacks added
row inf 1.77636e-15
column inf 1.07699e-09
row inf 0
column inf 0
row inf 1.13243e-14
column inf 9.82689e-09
198 slacks added
row inf 1.52101e-14
column inf 1.21139e-08
row inf 0
column inf 0
row inf 0
column inf 0
row inf 0
column inf 0
row inf 1.16573e-14
column inf 1.38869e-08
1 slacks added
1 slacks added
1 slacks added
1 slacks added
1 slacks added
1 slacks added
row inf 1.53211e-14
column inf 1.32399e-08
1 slacks added
row inf 0
column inf 0
67 slacks added
174 slacks added
304 slacks added
498 slacks added
608 slacks added
400 slacks added
613 slacks added
323 slacks added
row inf 0
column inf 0
row inf 1.34337e-14
column inf 2.79961e-08
49 slacks added
row inf 0
column inf 0
row inf 1.65423e-14
column inf 1.7241e-08
row inf 0
column inf 0


In [3]:
# Criando um dataframe com os dados coletados
const dataFrame = DataFrame(
    file_name = fileName,
    agents_number = agentsNumber,
    solve_time = solveTime,
    is_optimal_solution = isOptimalSolution,
    solution = solution
)

# Escrevendo esse dataframe em um .csv para uma futura análise dos dados
CSV.write("cbc_data_insta.csv", dataFrame)

println(dataFrame)

105×5 DataFrame
 Row │ file_name                          agents_number  solve_time  is_optimal_solution  solution 
     │ Any                                Any            Any         Any                  Any      
─────┼─────────────────────────────────────────────────────────────────────────────────────────────
   1 │ instancias/insta/inst_1_100.txt    100            0.339009    true                 100.0
   2 │ instancias/insta/inst_1_1000.txt   1000           181.141     true                 1000.0
   3 │ instancias/insta/inst_1_200.txt    200            0.956972    true                 200.0
   4 │ instancias/insta/inst_1_50.txt     50             0.0360661   true                 50.0
   5 │ instancias/insta/inst_1_500.txt    500            19.4373     true                 500.0
   6 │ instancias/insta/insta_2_10_100.…  100            0.282089    true                 4972.0
   7 │ instancias/insta/insta_2_10_1000…  1000           56.0942     true                 50000.0
   8 │ in